<a href="https://colab.research.google.com/github/ReynaQuita/NLP/blob/main/Translation_mBART25_wo_ft_enro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets transformers sacrebleu

     |████████████████████████████████| 264 kB 4.2 MB/s 
     |████████████████████████████████| 2.6 MB 62.3 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 118 kB 78.9 MB/s 
     |████████████████████████████████| 76 kB 5.1 MB/s 
     |████████████████████████████████| 243 kB 76.0 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 636 kB 71.4 MB/s 
     |████████████████████████████████| 895 kB 60.2 MB/s 
     |████████████████████████████████| 3.3 MB 56.3 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
model_checkpoint = "facebook/mbart-large-en-ro"

In [3]:
from datasets import load_dataset, load_metric

# train_datasets = load_dataset("wmt16", "ro-en", split="train[:1%]")
# val_datasets = load_dataset("wmt16", "ro-en", split="validation[:5%]")
metric = load_metric("sacrebleu")

Downloading:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [6]:
test_data = load_dataset("wmt16", "ro-en", split="test[:5%]")
len(test_data)

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/225M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/ro-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a. Subsequent calls will reuse this data.


100

In [7]:
if "mbart" in model_checkpoint:
    tokenizer.src_lang = "en_XX"
    tokenizer.tgt_lang = "ro_RO"

In [8]:
def generate_translation(batch):
    # cut off at BERT max length 512

    english = [ex["en"] for ex in batch["translation"]]

    inputs = tokenizer(english, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids

    attention_mask = inputs.attention_mask

    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True, forced_bos_token_id = tokenizer.lang_code_to_id["ro_RO"])

    batch["pred_translation"] = output_str

    return batch

In [9]:
batch_size = 4  # change to 64 for full evaluation

results = test_data.map(generate_translation, batched=True, batch_size=batch_size)

  0%|          | 0/25 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [10]:
bleu_score = 0

for i in range(len(results)):
  ref = results["translation"][i]["ro"]
  pred = results["pred_translation"][i]

  bleu_metric = metric.compute(predictions=[pred], references=[[ref]])
  bleu_score  += bleu_metric["score"]


In [11]:
print('Bleu Score: {} (scale 0-100)'.format(bleu_score/len(results)))

Bleu Score: 26.105280975413518 (scale 0-100)


In [12]:
results['translation'][:5]

[{'en': 'UN Chief Says There Is No Military Solution in Syria',
  'ro': 'Șeful ONU declară că nu există soluții militare în Siria'},
 {'en': 'Secretary-General Ban Ki-moon says his response to Russia\'s stepped up military support for Syria is that "there is no military solution" to the nearly five-year conflict and more weapons will only worsen the violence and misery for millions of people.',
  'ro': 'Secretarul General Ban Ki-moon afirmă că răspunsul său la suportul militar al Rusiei pentru Siria este că „nu există o soluție militară” la conflictul care durează de aproape cinci ani iar mai multe arme nu ar face decât să agraveze violența și suferința a milioane de oameni.'},
 {'en': 'The U.N. chief again urged all parties, including the divided U.N. Security Council, to unite and support inclusive negotiations to find a political solution.',
  'ro': 'Șeful ONU a solicitat din nou tuturor părților, inclusiv Consiliului de securitate ONU divizat să se unifice și să susțină negocierile

In [13]:
results['pred_translation'][:5]

['Şeful ONU declară că nu există o soluţie militară în Siria',
 'Secretarul General Ban Ki-moon declară că răspunsul său la intensificarea sprijinului militar al Rusiei pentru Siria este că "nu există o soluţie militară" la conflictul de aproape cinci ani şi că noi arme nu vor face decât să înrăutăţească violenţa şi mizeria pentru milioane de oameni.',
 'Şeful U.N. a cerut din nou tuturor părţilor, inclusiv Consiliului de Securitate divizat al U.N., să se unească şi să sprijine negocierile favorabile incluziunii pentru găsirea unei soluţii politice.',
 'Ban a declarat miercuri la o conferinţă de presă că intenţionează să se întâlnească cu miniştrii de externe ai celor cinci naţiuni permanente ale consiliului - SUA, Rusia, China, Marea Britanie şi Franţa - în paralel cu sesiunea ministerială a Adunării Generale din cursul acestei luni pentru a discuta despre Siria.',
 'El şi-a exprimat regretul că diviziunile din consiliu şi dintre poporul sirian şi puterile regionale "au făcut ca aceas